In [2]:
import pandas as pd
import numpy as np
from decimal import Decimal

In [3]:
file_path = r"C:\Users\shiva\Documents\Drive_through\Drive_Through_Dummy_Data.xlsx"
data = pd.read_excel(file_path)

In [5]:
data.columns

Index(['S_No.', 'Longitude', 'Latitude', 'Junction_n', 'Signal_bil',
       'District', 'Green_light_appe', 'Changing_time(in s)',
       'Speed_limit(in km/h)'],
      dtype='object')

In [18]:
data

,S_No.,Longitude,Latitude,Junction_n,Signal_bil,District,Green_light_appe,Changing_time(in s),Speed_limit(in km/h)
0,247,76.98847300,28.82895600,Auchndhi Border,With Pedestrian,Rohini,12:07:00,180,40
1,315,76.96603330,28.80618330,Katewra More,Blinker,Rohini,09:56:00,60,40
2,349,76.96174170,28.80057500,Village Jatkhore crossing Near Divya Dham Ashram,Blinker,Rohini,10:23:00,120,40
3,342,76.96266520,28.79040320,Sohati Road crossing in front of NDPL Office,Blinker,Rohini,16:04:00,120,40
4,249,77.00271800,28.81854800,Bawana Auchandi Road Harewali Road T-point,Blinker,Outer North,12:13:00,60,35
5,283,77.04946600,28.80812200,T Point CRPF CAMP,Blinker,Outer North,05:34:00,180,35
6,284,77.03518700,28.79751100,T- Point Kanjhawala bawana road badli,With Pedestrian,Outer North,02:14:00,120,35
7,257,77.02386330,28.78481000,Kanjhawala Road nangal thakran road t-point,Blinker,Outer North,08:37:00,60,35
8,348,77.01634900,28.76643300,T-point Majra Dabas Kanjhawala Bawana Road,Blinker,Rohini,06:58:00,60,40
9,314,77.01438800,28.75141200,Kanjhawala Chowk,Without Pedestrian,Rohini,01:03:00,60,40


In [6]:
data['Latitude'] = data['Latitude'].apply(lambda x: Decimal(str(x)).quantize(Decimal('1.00000000')))
data['Longitude'] = data['Longitude'].apply(lambda x: Decimal(str(x)).quantize(Decimal('1.00000000')))

In [7]:
def distance(lat1, lon1, lat2, lon2):
    R = 6371
    lat1, lon1, lat2, lon2 = map(np.radians, [float(lat1), float(lon1), float(lat2), float(lon2)])
    
    a = np.sin(lat1) * np.sin(lat2) + np.cos(lat1) * np.cos(lat2) * np.cos(lon2 - lon1)
    distance = np.arccos(a) * R
    
    return distance

In [8]:
distance(76.988473, 28.828956, 77.016349, 28.766433)

3.4717264580436633

In [19]:
def get_time(t):
    try:
        return pd.to_datetime(t, format='%H:%M:%S').time()
    except ValueError:
        return pd.to_datetime(t, format='%H:%M').time()

In [20]:
def path(start_lat, start_lon, end_lat, end_lon, data):
    min_lat, max_lat = min(start_lat, end_lat), max(start_lat, end_lat)
    min_lon, max_lon = min(start_lon, end_lon), max(start_lon, end_lon)
    
    print(f"Min Latitude: {min_lat}, Max Latitude: {max_lat}")
    print(f"Min Longitude: {min_lon}, Max Longitude: {max_lon}")
    
    path_points = []
    
    for i in range(len(data)):
        lat = Decimal(str(data.iloc[i]['Latitude'])).quantize(Decimal('1.00000000'))
        lon = Decimal(str(data.iloc[i]['Longitude'])).quantize(Decimal('1.00000000'))
        
        if lat.compare(min_lat) >= 0 and lat.compare(max_lat) <= 0 and lon.compare(min_lon) >= 0 and lon.compare(max_lon) <= 0:
            path_points.append(data.iloc[i])
    
    path_points = pd.DataFrame(path_points)
    print(f"Intermediate data points found: {len(path_points)}")
    
    return path_points

In [21]:
find_intermediate_points(77.0984433, 28.8567367,76.97872, 28.7458083,data)

Min Latitude: 76.97872000, Max Latitude: 77.09844330
Min Longitude: 28.74580830, Max Longitude: 28.85673670
Intermediate data points found: 0


""


In [16]:
def calculate_optimal_speed(start_lat, start_lon, end_lat, end_lon, data):
    intermediate_points = find_intermediate_points(start_lat, start_lon, end_lat, end_lon, data)
    optimal_speed = None
    optimal_path = []
    
    for speed in range(20, 100, 5):
        total_time = 0
        all_green = True
        path = []
        
        print(f"Testing speed: {speed} km/h")
        
        for idx, row in intermediate_points.iterrows():
            dist = distance(start_lat, start_lon, row['Latitude'], row['Longitude'])
            time_to_reach = (dist / speed) * 3600  # Time in seconds
            green_light_start = parse_time(row['Green_light_appe'])
            green_light_end = (pd.to_datetime(row['Green_light_appe'], format='%H:%M:%S') + pd.Timedelta(seconds=row['Changing_time(in s)'])).time()
            
            # Check if the time_to_reach falls within the green light window
            arrival_time = (pd.to_datetime('00:00:00') + pd.Timedelta(seconds=total_time + time_to_reach)).time()
            
            print(f"Dist: {dist:.2f} km, Time to reach: {time_to_reach:.2f} s, Arrival time: {arrival_time}, Green light window: {green_light_start} - {green_light_end}")
            
            if green_light_start <= arrival_time <= green_light_end:
                total_time += time_to_reach
                start_lat, start_lon = row['Latitude'], row['Longitude']  # Update starting point to the current junction
                path.append(row['Junction_n'])  # Append the junction name to the path
            else:
                all_green = False
                break
        
        if all_green:
            optimal_speed = speed
            optimal_path = path
            break
    
    return optimal_speed, optimal_path

In [17]:
# Example Usage
start_lat, start_lon = 28.828956, 76.988473 
end_lat, end_lon = 28.841083, 77.1089  
optimal_speed, optimal_path = calculate_optimal_speed(start_lat, start_lon, end_lat, end_lon, data)

print(f"The optimal speed to ensure all green signals is: {optimal_speed} km/h")
print(f"The corresponding junction points are: {optimal_path}")

Min Latitude: 28.82895600, Max Latitude: 28.84108300
Min Longitude: 76.98847300, Max Longitude: 77.10890000
Intermediate data points found: 3
Testing speed: 20 km/h
Dist: 0.00 km, Time to reach: 0.00 s, Arrival time: 00:00:00, Green light window: 12:07:00 - 12:10:00
Testing speed: 25 km/h
Dist: 0.00 km, Time to reach: 0.00 s, Arrival time: 00:00:00, Green light window: 12:07:00 - 12:10:00
Testing speed: 30 km/h
Dist: 0.00 km, Time to reach: 0.00 s, Arrival time: 00:00:00, Green light window: 12:07:00 - 12:10:00
Testing speed: 35 km/h
Dist: 0.00 km, Time to reach: 0.00 s, Arrival time: 00:00:00, Green light window: 12:07:00 - 12:10:00
Testing speed: 40 km/h
Dist: 0.00 km, Time to reach: 0.00 s, Arrival time: 00:00:00, Green light window: 12:07:00 - 12:10:00
Testing speed: 45 km/h
Dist: 0.00 km, Time to reach: 0.00 s, Arrival time: 00:00:00, Green light window: 12:07:00 - 12:10:00
Testing speed: 50 km/h
Dist: 0.00 km, Time to reach: 0.00 s, Arrival time: 00:00:00, Green light window: 12:

In [22]:
import math

def calcBearing (lat1, long1, lat2, long2):
    dLon = (long2 - long1)
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(dLon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(dLon))
    bearing = math.atan2(x,y)   # use atan2 to determine the quadrant
    bearing = math.degrees(bearing)

    return bearing

def calcNSEW(lat1, long1, lat2, long2):
    points = ["north", "north east", "east", "south east", "south", "south west", "west", "north west"]
    bearing = calcBearing(lat1, long1, lat2, long2)
    bearing += 22.5
    bearing = bearing % 360
    bearing = int(bearing / 45) # values 0 to 7
    NSEW = points [bearing]

    return NSEW

# White house 38.8977° N, 77.0365° W
lat1 = 38.8976763
long1 = -77.0365298
# Lincoln memorial 38.8893° N, 77.0506° W
lat2 = 38.8893
long2 = -77.0506

points = calcNSEW(lat1, long1, lat2, long2)
print ("The Lincoln memorial is " + points + " of the White House")
print ("Actually bearing of 231.88 degrees")

print ("Output says: The Lincoln memorial is south west of the White House ")


The Lincoln memorial is south west of the White House
Actually bearing of 231.88 degrees
Output says: The Lincoln memorial is south west of the White House 
